## Problem
Everytime mean value of the data columns keep changing

In [2]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 


In [161]:
#display(data)
def filter_data(data):
    
    max_flux = 12
    min_flux = 26
    data = data[data['flux_aper']>max_flux]

    data = data[data['significance']>2]
    data = data.sample(frac=1)
    data_class = data[['class']]
    data_sig = data['significance']
    obs_info_params = [ 'livetime','significance','likelihood','pileup_flag','mstr_sat_src_flag','mstr_streak_src_flag'   ,'gti_obs' , 'flux_significance_b'  , 'flux_significance_m' , 'flux_significance_s' , 'flux_significance_h' , 'flux_significance_u'    ]
    data_val = data.drop(columns=obs_info_params)
    flux_params = ['photflux_aper' ,'photflux_aper_lolim' , 'photflux_aper_hilim' , 'flux_aper' , 'flux_aper_lolim' , 'flux_aper_hilim' ]
    flux_params_band = []
    filters = ['_b' ,'_h' ,'_m' ,'_u' ,'_s']
    for fp in flux_params:
        for f in filters:
            temp = fp+f
            flux_params_band.append(temp)

    data_val = data_val.drop(columns=['index' , 'class'])

    import missingno as msno

    data_val = data_val.replace(np.nan , 0)
    for d in data_val:
        mean = np.mean(data_val[d])
        var = np.var(data_val[d])
        #data_val[d] = (data_val[d]-mean)/var
        data_val[d] = data_val[d] / np.amax(data_val[d])
    return data , data_val , data_class , data_sig 

train_bh = pd.read_csv('processed_data/BH_.csv').reset_index(drop=True)
train_ns = pd.read_csv('processed_data/NS_.csv').reset_index(drop=True)
train = pd.concat([train_bh , train_ns ] , axis=0)
#train =  train.sample(frac=1)

train_data ,  data_val , data_class , data_sig = filter_data(train)
display(data_val.describe().iloc[:,10:])

,flux_aper_s,flux_aper_u,flux_aper_m,flux_aper_h,flux_aper_b,photflux_aper_hilim_s,photflux_aper_hilim_u,photflux_aper_hilim_m,photflux_aper_hilim_h,photflux_aper_hilim_b,...,flux_brems,flux_brems_lolim,flux_brems_hilim,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
count,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,...,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000,460.000000
mean,0.600407,0.169058,0.647555,0.715904,0.801578,0.731254,0.632244,0.725824,0.750497,0.718758,...,0.023757,0.022649,0.024260,0.011096,0.015854,0.006952,0.011490,0.012258,0.010286,0.017480
std,0.428391,0.351225,0.404720,0.344085,0.242056,0.200449,0.315364,0.187184,0.124476,0.142139,...,0.109221,0.105525,0.110595,0.085123,0.078341,0.052841,0.075177,0.080678,0.068566,0.066559
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.813599,0.824962,0.680457,0.562745,0.673480,0.688801,0.661747,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.865366,0.000000,0.874249,0.862341,0.868969,0.784904,0.737185,0.773367,0.746661,0.741014,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.930838,0.000000,0.923043,0.898844,0.906662,0.840864,0.866017,0.831748,0.841255,0.801781,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
flux = data_val[['flux_aper']]
display(flux)

,flux_aper
152,-0.487623
337,-0.539722
154,-0.277405
193,0.135965
363,0.451175
...,...
133,-0.327574
32,-0.601488
18,0.067456
323,-0.314309


In [152]:
flux = flux.sample(frac=1)
flux.describe()

,flux_aper
count,4.600000e+02
mean,-8.721285e-16
std,3.503408e-01
min,-9.202868e-01
25%,-2.261461e-01
50%,-7.115291e-03
75%,2.425957e-01
max,1.000000e+00


## Conclusion
Not a problem actually , since we are normalising distribution and making mean go to zero , so the mean is actually close to zero , and the value displayed is actually garbage value